In [1]:
import pandas as pd
import pickle

In [2]:
with open("/home/luca/ml-malware-concept-drift/data/dataset/dataset.pickle", "rb") as f:
    df = pickle.load(f)

In [3]:
df.shape

(67000, 42231)

In [4]:
df.head()

,generic_fileSize,generic_fileEntropy,header_SizeOfHeaders,header_AddressOfEntryPoint,header_ImageBase,header_SizeOfImage,header_SizeOfCode,header_SizeOfInitializedData,header_SizeOfUninitializedData,header_BaseOfCode,...,pesection_12_exists,pesection_12_isStandard,pesection_13_exists,pesection_13_isStandard,pesection_14_exists,pesection_14_isStandard,pesection_15_exists,pesection_15_isStandard,pesection_16_exists,pesection_16_isStandard
sample_hash,,,,,,,,,,,,,,,,,,,,,
cfa2d5eeac7de9f134cb2e36d54a5c4053f62fdf9d5302994557da1287719414,1666137,7.996651,1024,5358807,4194304,5365760,310272,56832,0,4096,...,True,False,False,False,False,False,False,False,False,False
00ebebc75f61527282cee19ab7aed80693b63fbb969e7154d17838f9dd08db2f,10543104,4.821304,4096,4512,4194304,118784,94208,20480,0,4096,...,False,False,False,False,False,False,False,False,False,False
f45e6ca3a2bbbaa1c514d81abe9daaa47f7d41da500e6933f61adf29f5f39835,57368,4.979988,4096,9759,4194304,57344,8192,45056,0,4096,...,False,False,False,False,False,False,False,False,False,False
d694a203bb211751669f8742db877e8ebd5eff5b126abc4c3e59cdb81f9dbb56,7922688,7.950081,1024,155508,4194304,7950336,570880,7350784,0,4096,...,False,False,False,False,False,False,False,False,False,False
15c9feb810c48ffef1c7e32cbe58a11037da94e958fb1f34e8feca1643dd80eb,159744,5.911521,4096,86524,4194304,159744,86016,69632,0,4096,...,False,False,False,False,False,False,False,False,False,False


In [5]:
len(set(list(df.index)))

67000

In [6]:
# Check if any column is of type object
is_bool_column = df.dtypes == 'object'
obj_cols = df.columns[is_bool_column]
obj_cols

Index(['pesectionProcessed_entrypointSection_name'], dtype='object')

In [7]:
df = df.drop(obj_cols, axis=1)

In [8]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
from sklearn import random_projection
from sklearn.model_selection import ParameterGrid

In [9]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram
    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [10]:
# X = df
# transformer = random_projection.GaussianRandomProjection()
# X_new = transformer.fit_transform(X)
# X_new.shape

In [11]:
#X_new.to_csv("dataset_rp.csv")
#pd.DataFrame(X_new).to_csv("dataset_rp.csv")
X_new = pd.read_csv("dataset_rp.csv")
X_new.shape

(67000, 9525)

In [12]:
param_grid = {'linkage': ['ward', 'complete', 'average', 'single']}
#sil_thr = 0  # visualize results only for combinations with silhouette above the threshold
#unc_thr = 10 # visualize results only for combinations with unclustered% below the threshold
params = list(ParameterGrid(param_grid))

In [13]:
for i in range(len(params)):
    # setting distance_threshold=0 ensures we compute the full tree.
    model = AgglomerativeClustering(distance_threshold=0, n_clusters=None, **(params[i]))
    model = model.fit(X_new)
    plt.title(f"Hierarchical Clustering Dendrogram {params[i]}")
    # plot the top ten levels of the dendrogram
    plot_dendrogram(model, truncate_mode="level", p=10)
    plt.xlabel("Number of points in node (or index of point if no parenthesis).")
    plt.show()

NameError: name 'plt' is not defined